In [ ]:
def tokenize(token, category, tokens):
  if len(token) == 0 or category == "whitespace" or "\n" in token or " " in token:
    pass
  elif category == "letter":
    tokenize_letter(token, tokens)
  else:
    tokens.append({"token": token, "category": category, "row": get_row(), "column": get_column()})

In [ ]:
def get_row():
  global row
  return row

def get_column():
  global col
  return col + 1

In [ ]:
def tokenize_letter(token, tokens):
  RESERVED_LIST = ['using', 'namespace', 'int', 'main', 'if', 'else', 'cin', 'cout', 'return', 'for', 'struct']
  if token in RESERVED_LIST:
    tokens.append({"token": token, "category": "Reserved word", "row": get_row(), "column": get_column()})
  else:
    tokens.append({"token": token, "category": "Variable or namespace", "row": get_row(), "column": get_column()})


In [ ]:
def tokenize_directiva(file, tokens):
  global col
  
  if col == 1:
    token = '#'
    c = 'q'
    while c != '\n':
      c = read_symbol(file)
      token += c
    token = token[:-1]
    tokens.append({"token": token, "category": "include", "row": get_row() - 1, "column": get_column()})
  
      


In [ ]:
def tokenize_string(file, tokens):
  token = ''
  c = 'q'
  while c != '"':
    c = read_symbol(file)
    token += c
  token = token[:-1]
  tokens.append({"token":token, "category": "String literal", "row": get_row(), "column": get_column()})
  tokens.append({"token":'"', "category": "double quanties", "row": get_row(), "column": get_column()})

In [ ]:
def tokenize_char(file, tokens):
  token = ''
  c = 'q'
  while c != '"':
    c = read_symbol(file)
    token += c
  token = token[:-1]
  tokens.append({"token":token, "category": "char literal", "row": get_row(), "column": get_column()})
  tokens.append({"token":'"', "category": "single quanties", "row": get_row(), "column": get_column()})

In [ ]:
def get_category(c):
  if c.isnumeric():
    return "number"
  if c == '#':
    return "Directiva"
  if c.isalpha():
    return "letter"
  if c == ' ':
    return "whitespace"
  if c == ';':
    return "operator split"
  if c == ',':
    return 'comma'
  if c == '.':
    return 'dot'
  if c in "([{":
    return "open bracket"
  if c in ")]}":
    return "close bracket"
  if c in "+/=<>-%":
    return "operators"
  if c == '"':
    return 'double quanties'
  if c == "'":
    return 'single quanties'
  return 'Unknown'

In [ ]:
def read_symbol(file):
  global pos, row, col
  c = file.read(1)
  
  if  c == '\n':
    row += 1
    col = 0
  elif c:
    col += 1
  else:
    return 0
  return c


In [ ]:
operators = ['=', '<', '>', '+', '-', '<<', '==', '--', '++', '/']
def errorizer(lines, tokens):
  print("ERRORS")
  for i in range(1, len(tokens)):
    if tokens[i]['category'] == 'operators' and tokens[i]['token'] not in operators:
      print(f"line {tokens[i]['row']} char {tokens[i]['column']} :: Неизвестный оператор")
      print(lines[tokens[i]['row'] - 1][:-1])
      print(' ' *  tokens[i]['column'] + '^')
    elif tokens[i]['category'] == tokens[i - 1]['category'] and tokens[i]['category'] != 'Reserved word':
      print(f"line {tokens[i - 1]['row']} char {tokens[i - 1]['column']} :: expected initializer before '{tokens[i - 1]['token']}'")
      print(lines[tokens[i]['row'] - 1][:-1])
      print(' ' * tokens[i - 1]['column'] + '^')
    
  for i in range(len(lines)):
    if len(lines[i]) == 1:
      continue
    if lines[i][-2] == ';' and (lines[i][-3] == '{' or lines[i][-3] == '}' or lines[i][-3] == '>'):
      print(f"line {i + 1} char {len(lines[i])} :: Unxpected ';'")
      print(lines[tokens[i]['row']][:-1])
      print(' ' * (len(lines[i]) - 2) + '^')
    if lines[i][-2] != ';' and (lines[i][-3] != '{' and lines[i][-3] != '}' and lines[i][-3] != '>'):
      print(f"line {i + 1} char {len(lines[i])} :: Expected ';'")
      print(lines[tokens[i]['row']][:-1])
      print(' ' * (len(lines[i]) - 2) + '^')


    

In [ ]:
col, row = 0, 1
file = open("main.cpp")
f = open("main.cpp")
Lines = f.readlines()
tokens = []
pattern = ""
pred_category, category = "", ""
while True:
  c = read_symbol(file)
  if c == 0:
    break
  pred_category = category
  category = get_category(c)
  if category == "Directiva":
    tokenize_directiva(file, tokens)
    continue
  elif category == "double quanties":
    tokenize_string(file, tokens)
    continue
  elif category == "single quanties":
    tokenize_char(file, tokens)
    continue
  elif category == "":
    tokenize_string(file, tokens)
  
  elif pred_category != category:
    pass
    tokenize(pattern, pred_category, tokens)
    pattern = ""
  pattern += c
errorizer(Lines, tokens)    
categories = []
for token in tokens:
  categories.append(token['category'])
categories = set(categories)
print('TOKENS')
for category in categories:
  print("_____________")
  print(f"CATEGORY: {category}")
  for token in tokens:
    if token['category'] == category:
      print(f"TOKEN: {token['token']} POSITION: ({token['row']}, {token['column']})")

1
ERRORS
line 1 char 19 :: Expected ';'
using namespace std;
                 ^
line 3 char 13 :: Unxpected ';'
int main(){;
           ^
line 10 char 18 :: Expected ';'
  int a[10] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
                ^
line 16 char 4 :: Expected ';'
  int n = 10, key = 5;
  ^
line 20 char 60 :: Expected ';'
  int n = 10, key = 5;
                                                          ^
TOKENS
_____________
CATEGORY: operators
TOKEN: = POSITION: (4, 15)
TOKEN: = POSITION: (5, 11)
TOKEN: = POSITION: (5, 21)
TOKEN: = POSITION: (6, 11)
TOKEN: = POSITION: (7, 11)
TOKEN: < POSITION: (10, 14)
TOKEN: = POSITION: (11, 11)
TOKEN: + POSITION: (11, 16)
TOKEN: / POSITION: (11, 21)
TOKEN: > POSITION: (13, 18)
TOKEN: = POSITION: (13, 27)
TOKEN: = POSITION: (14, 14)
TOKEN: + POSITION: (14, 20)
TOKEN: -- POSITION: (17, 7)
TOKEN: == POSITION: (19, 15)
TOKEN: << POSITION: (19, 28)
TOKEN: << POSITION: (19, 50)
TOKEN: << POSITION: (19, 57)
TOKEN: << POSITION: (19, 81)
TOKEN: << POSITION: